In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import h5py
from pathlib import Path
import os
import sys
sys.path.append(os.getcwd() + '/../..')
sys.path.append(os.getcwd() + '/..')
from filepaths import filepath_from_ml_artifacts_uri
from analysis.iclr.plotting import settings, fig_size, cb_line_cycler, cb_line_cycler_solid, cb_marker_cycler, savefig
from plot_hdf5_dataset import plot_dataset
from iclr_paper_path import get_paper_path
import matplotlib
settings()
import matplotlib.pyplot as plt

In [2]:
path_runs = 'runs_iclr_shf.csv'
df = pd.read_csv(path_runs)
for col in df.columns:
    print(col)

Start Time
Duration
Run ID
Name
Source Type
Source Name
User
Status
activation
alpha_mu_override
alpha_sigma_override
batch_size_override
batch_size_test
batches_per_epoch_override
beta1_adam_override
beta2_adam_override
beta_start_override
clip_grad_norm_override
control_encoder_linear
controls_to_decoder
dataset_name
decoder_linear
early_stopping_patience_override
early_stopping_threshold_mode_override
early_stopping_threshold_override
evaluate_at_control_times_override
hidden_dim_output_nn
include_reconstruction_loss_state0_override
initialization_type
initialization_type_ode
initialization_type_ode_matrix
job 1 ended by
job 2 ended by
job 3 ended by
koopman_mpc_mode
lat_controls_dim
lat_ode_type
lat_parameters_dim
lat_state_mu_independent
lat_states_dim
linear_hidden_dim
load_pretrained_model
load_seq_len_override
load_trained_model_for_test
lr_start_override
main_training
main_training_0_activate_deterministic_mode_after_this_phase
main_training_0_alpha_mu
main_training_0_alpha_si

In [3]:
def get_dataset_paths(lat_ode_type):
    df_lat = df[df['lat_ode_type'] == lat_ode_type]
    run_ids = df_lat['Run ID']
    paths = []
    betas = []
    n_states = []
    for run_id in run_ids:
        path = 'mlflow-artifacts:/829789035554334690/' + run_id + '/artifacts/dataset.hdf5'
        paths.append(path)
        betas.append(df_lat[df_lat['Run ID'] == run_id]['beta_start_override'].values[0])
        n_states.append(df_lat[df_lat['Run ID'] == run_id]['lat_dim_states_populated_common_test'].values[0] if lat_ode_type != 'vanilla' else 16)
    return paths, betas, n_states

_paths, betas, n_states = get_dataset_paths('variance_constant')
print(_paths)
print(betas)
print(n_states)

['mlflow-artifacts:/829789035554334690/f1f7fccda995499cada9e7f2d96b4026/artifacts/dataset.hdf5', 'mlflow-artifacts:/829789035554334690/0a57c3a169ef4071bcab5257c5fd88e6/artifacts/dataset.hdf5', 'mlflow-artifacts:/829789035554334690/e259efcef6be4ad8a29a85e103368901/artifacts/dataset.hdf5', 'mlflow-artifacts:/829789035554334690/3924d1144ac8453d99d9ce5177125623/artifacts/dataset.hdf5', 'mlflow-artifacts:/829789035554334690/c5a78b70ecf5451badad160ca9e6be2e/artifacts/dataset.hdf5', 'mlflow-artifacts:/829789035554334690/3949346e33e748aaa52c9c86bee5976e/artifacts/dataset.hdf5', 'mlflow-artifacts:/829789035554334690/2c1dfa85c8db4a6db742c68d1308e392/artifacts/dataset.hdf5', 'mlflow-artifacts:/829789035554334690/2ca1dc3ab4ca4fda96dc072fea11451f/artifacts/dataset.hdf5']
[0.001, 0.0, 1.0, 0.01, 0.04, 0.06, 0.08, 0.1]
[13, 16, 1, 5, 4, 4, 4, 4]


In [4]:
one_temp = False
path_ROM_dataset = Path('data\StratifiedHeatFlowModel_v3_c-RROCS__n-2048_common_test_ROM_dataset.hdf5')
ROM_dataset = h5py.File(path_ROM_dataset, 'r')

# path_LODE_dataset_beta0_01 = filepath_ml_artifacts_from_mlflow('mlflow-artifacts:/132556430175136789/a9cde693eb7447d988a453dee6a03e02/artifacts/dataset.hdf5')
# LODE_dataset_beta_0_01 = h5py.File(path_LODE_dataset_beta0_01, 'r')

# path_LODE_dataset_beta0_1 = filepath_ml_artifacts_from_mlflow('mlflow-artifacts:/132556430175136789/30d16f3ea36a437291d3a4ad212b2719/artifacts/dataset.hdf5')
# LODE_dataset_beta_0_1 = h5py.File(path_LODE_dataset_beta0_1, 'r')

# calculate mean and variance of states for normalization
var_states = ROM_dataset['states'][:].std(axis=(0, 2))
mean_states = ROM_dataset['states'][:].mean(axis=(0, 2))

# states = ROM_dataset['states'][:]
states_from_ROM = ROM_dataset['states'][:]

def calculate_normed_loss(states, states_hat, var_states=var_states, mean_states=mean_states, one_temp=one_temp):

    var_states = np.expand_dims(var_states, axis=(0, 2))
    mean_states = np.expand_dims(mean_states, axis=(0, 2))

    x = np.divide(states, mean_states)
    xhat = np.divide(states_hat, mean_states)

    # calculate rmse loss
    residual = x - xhat if not one_temp else x[:, 7, :] - xhat[:, 7, :]
    if one_temp:
        residual = np.expand_dims(residual, axis=1)
    loss = np.square(residual).mean()
    loss = np.sqrt(loss)

    return loss *100

# loss_LODE_beta0_01 = calculate_normed_loss(states, LODE_dataset_beta_0_01['common_test']['states_hat'][:])
# print('LODE loss: ', loss_LODE_beta0_01.mean())
# loss_LODE_beta0_1 = calculate_normed_loss(states, LODE_dataset_beta_0_1['common_test']['states_hat'][:])
# print('LODE loss: ', loss_LODE_beta0_1.mean())

loss_ROM = {}
for i in range(1,17):
    loss_ROM[i] = calculate_normed_loss(states_from_ROM, ROM_dataset['x_r-{}'.format(i)]['states_hat'][:])
    print('ROM loss r-{}: {:3f}'.format(i, loss_ROM[i]))
n_states = [i for i in range(1,17)]
_dict = {'n_states': n_states, 'loss': [loss_ROM[i] for i in range(1,17)]}
df_ROM = pd.DataFrame(_dict)

_paths, _betas, n_states = get_dataset_paths('variance_constant')
loss_BNODE_constant = []
betas_BNODE_constant = _betas
for i, path in enumerate(_paths):
    dataset = h5py.File(filepath_from_ml_artifacts_uri(path), 'r')
    states = dataset['common_test']['states'][:]
    rmse = calculate_normed_loss(states, dataset['common_test']['states_hat'][:])
    loss_BNODE_constant.append(rmse)
    print('BNODE const. beta {}: {:3f}'.format(betas_BNODE_constant[i], rmse))
_dict = {'n_states': n_states, 'loss': loss_BNODE_constant, 'betas': betas_BNODE_constant}
df_BNODE_constant = pd.DataFrame(_dict)

_paths, _betas, n_states = get_dataset_paths('variance_dynamic')
loss_BNODE_dynamic = []
betas_BNODE_dynamic = _betas
for i, path in enumerate(_paths):
    dataset = h5py.File(filepath_from_ml_artifacts_uri(path), 'r')
    states = dataset['common_test']['states'][:]
    rmse = calculate_normed_loss(states, dataset['common_test']['states_hat'][:])
    loss_BNODE_dynamic.append(rmse)
    print('BNODE var. beta {}: {:3f}'.format(betas_BNODE_dynamic[i], rmse))
_dict = {'n_states': n_states, 'loss': loss_BNODE_dynamic, 'betas': betas_BNODE_dynamic}
df_BNODE_dynamic = pd.DataFrame(_dict)

_paths, _betas, n_states = get_dataset_paths('vanilla')
loss_vanilla = []
betas_vanilla = _betas
for i, path in enumerate(_paths):
    dataset = h5py.File(filepath_from_ml_artifacts_uri(path), 'r')
    states = dataset['common_test']['states'][:]
    rmse = calculate_normed_loss(states, dataset['common_test']['states_hat'][:])
    loss_vanilla.append(rmse)
    print('Vanilla beta {}: {:3f}'.format(betas_vanilla[i], rmse))
_dict = {'n_states': n_states, 'loss': loss_vanilla, 'betas': betas_vanilla}
df_vanilla = pd.DataFrame(_dict)

# select best beta for each number of states
df_BNODE_constant = df_BNODE_constant.sort_values('loss').groupby('n_states').first().reset_index()
df_BNODE_dynamic = df_BNODE_dynamic.sort_values('loss').groupby('n_states').first().reset_index()

ROM loss r-1: 36.826734
ROM loss r-2: 36.515446
ROM loss r-3: 14.371049
ROM loss r-4: 14.218948
ROM loss r-5: 4.210234
ROM loss r-6: 4.144241
ROM loss r-7: 0.940614
ROM loss r-8: 0.920287
ROM loss r-9: 0.154338
ROM loss r-10: 0.149537
ROM loss r-11: 0.021130
ROM loss r-12: 0.020360
ROM loss r-13: 0.012451
ROM loss r-14: 0.012456
ROM loss r-15: 0.012524
ROM loss r-16: 0.012581
BNODE const. beta 0.001: 0.154541
BNODE const. beta 0.0: 0.141584
BNODE const. beta 1.0: 2.332313
BNODE const. beta 0.01: 0.201923
BNODE const. beta 0.04: 0.321480
BNODE const. beta 0.06: 0.333900
BNODE const. beta 0.08: 0.424841
BNODE const. beta 0.1: 0.436902
BNODE var. beta 0.001: 7.462342
BNODE var. beta 1.0: 2.304604
BNODE var. beta 0.01: 0.136657
BNODE var. beta 0.04: 0.297895
BNODE var. beta 0.06: 0.390014
BNODE var. beta 0.08: 0.380768
BNODE var. beta 0.1: 0.378091
Vanilla beta 0.1: 0.456122


In [11]:
df_ROM

,n_states,loss
0,1,36.826734
1,2,36.515446
2,3,14.371049
3,4,14.218948
4,5,4.210234
5,6,4.144241
6,7,0.940614
7,8,0.920287
8,9,0.154338
9,10,0.149537


In [5]:
df_BNODE_constant

,n_states,loss,betas
0,1,2.332313,1.000
1,4,0.321480,0.040
2,5,0.201923,0.010
3,13,0.154541,0.001
4,16,0.141584,0.000


In [6]:
df_BNODE_dynamic

,n_states,loss,betas
0,0,7.462342,0.001
1,1,2.304604,1.000
2,4,0.297895,0.040
3,5,0.136657,0.010


In [10]:
fig, ax = plt.subplots(1, 1, figsize=fig_size(1.5, n_figs_per_width=2), tight_layout=True, )
# adjust subplot
plt.subplots_adjust(left=0.0, right=1.0, top=1.0, bottom=0.0)
# set marker cycler
ax.set_prop_cycle(cb_marker_cycler)

# x = [4,5]
# y = [loss_LODE_beta0_1.mean(), loss_LODE_beta0_01.mean()]
# ax.plot(x,y, label='Latent State Space ODE')
# for x,y,beta in zip(x,y, [0.1, 0.01]):
#     ax.text(x + 0.40, y, r'$\beta={}$'.format(beta))
ax.plot(df_BNODE_constant['n_states'], df_BNODE_constant['loss'], label=r'B-NODE, $\sigma$ const.', markersize=5, markeredgewidth=1)
for x,y,beta in zip(df_BNODE_constant['n_states'], df_BNODE_constant['loss'], df_BNODE_constant['betas']):
    ax.text(x + 0.20, 1.6* y, r'$\beta={}$'.format(beta), fontsize=3, color = '#E69F00', ha ='right', rotation = 90)

ax.plot(df_BNODE_dynamic['n_states'], df_BNODE_dynamic['loss'], label=r'B-NODE, $\sigma$ dyn.', markersize=5, markeredgewidth=1)
for x,y,beta in zip(df_BNODE_dynamic['n_states'], df_BNODE_dynamic['loss'], df_BNODE_dynamic['betas']):
    ax.text(x + 0.20, 1.6 * y, r'$\beta={}$'.format(beta), fontsize=3, color = '#56B4E9', ha = 'left', rotation = 90)

ax.plot(df_vanilla['n_states'], df_vanilla['loss'], label='Latent ODE', markersize=5, markeredgewidth=1)
# for x,y,beta in zip(df_vanilla['n_states'], df_vanilla['loss'], df_vanilla['betas']):
#     ax.text(x + 0.40, y, r'$\beta={}$'.format(beta))

ax.plot(df_ROM['n_states'], df_ROM['loss'], label='TBR', markersize=5, markeredgewidth=1)

ax.set_yscale('log')
ax.set_ylim([0.01, 50])
x_ticks = [np.linspace(0.01, 0.1, 10), np.linspace(0.1, 1, 10), np.linspace(1, 10, 10), np.linspace(10, 100, 10)]
x_ticks = np.concatenate(x_ticks)
ax.set_yticks(x_ticks, minor=True)
# show minor ticks without label
ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

# place x ticks at even numbers
# ax.set_xticks(np.arange(0, 17, 2))
ax.set_xticks(np.arange(0, 17, 2))

# add finer y ticks for log scale
# ax.yaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())
# ax.yaxis.set_minor_formatter(matplotlib.ticker.ScalarFormatter())


ax.set_ylabel(r'RMSE [\%]')
ax.set_xlabel(r'Number of states')
# ax.set_ylim([5e-6, 5e3])
# place legend above plot
ax.legend(loc='lower center', ncol=2, shadow=False, frameon=True, bbox_to_anchor=(0.5, 1.0), )
ax.grid(which='both', linewidth=0.2)
savefig(fig, get_paper_path('BNODE', 'rmse_vs_n_states'))

finding path that contains "code", then going up one level to find "LaTeX"
e:\projects\balanced_neural_odes_internal\code\python\analysis\iclr
e:\projects\balanced_neural_odes_internal\code\python\analysis
e:\projects\balanced_neural_odes_internal\code\python
e:\projects\balanced_neural_odes_internal\code
e:\projects\balanced_neural_odes_internal
E:\projects\balanced_neural_odes_internal\LaTeX\ICLR 2025 Template\figures\BNODE\rmse_vs_n_states.png
E:\projects\balanced_neural_odes_internal\LaTeX\ICLR 2025 Template\figures\BNODE\rmse_vs_n_states.svg
